# Record a snapshot of PV values

See https://github.com/BCDA-APS/apstools/issues/62

At certain times, we need a snapshot of certain PV values for later review and comparison. This is a perfect job for a Bluesky plan, to record the snapshot and save it in the databroker. Later, standard databroker tools can be use to retrieve the snapshot. This is aided by writing certain metadata with each snapshot.

In [1]:
# first the imports

import os
import time
from collections import OrderedDict
# import pyRestTable
import datetime

# Import matplotlib and put it in interactive mode.
%matplotlib notebook
import matplotlib.pyplot as plt
plt.ion()

from ophyd import EpicsSignal
from databroker import Broker
from bluesky import RunEngine
from bluesky import plan_stubs as bps

from bluesky.utils import install_nb_kicker
install_nb_kicker()

## snapshot()

The core method here is the Bluesky plan to record a snapshot.  It receives a list of ophyd `Signal` objects and records their values to the primary data data stream in a single event.  Some additional metadata is added to make this searchable from the databroker.

In [2]:
def snapshot(obj_list, stream="primary", md=None):
    """
    bluesky plan: record current value of list of ophyd signals
    """
    # we want this metadata to appear
    _md = dict(
        plan_name = "snapshot",
        plan_description = "archive snapshot of ophyd Signals (usually EPICS PVs)",
        iso8601 = str(datetime.datetime.now()),     # human-readable
        hints = {},
        )
    # caller may have given us additional metadata
    _md.update(md or {})

    def _snap(md=None):
        yield from bps.open_run(md)
        yield from bps.create(name=stream)
        for obj in obj_list:
            # passive observation: DO NOT TRIGGER, only read
            yield from bps.read(obj)
        yield from bps.save()
        yield from bps.close_run()

    return (yield from _snap(md=_md))

## creating the ophyd objects

We want the user to provide a list of EPICS PV names, from which a list of ophyd `EpicsSignal` objects is created.  For this demo, we have an [IOC](https://github.com/prjemian/epics_ioc_raspi_sensors#epics-ioc-on-raspberry-pi-with-gpio-sensors) with a [DHT-22](https://www.sparkfun.com/datasheets/Sensors/Temperature/DHT22.pdf) humidity and temperature sensor.

In [3]:
pvlist = """
rpi5bf5:0:humidity
rpi5bf5:0:temperature
rpi5bf5:0:humidity:1m
rpi5bf5:0:temperature:1m
rpi5bf5:0:humidity:1m:sdev
rpi5bf5:0:temperature:1m:sdev
""".strip().splitlines()

def connect_pvlist(pvlist):
    obj_dict = OrderedDict()
    for item in pvlist:
        if len(item.strip()) == 0:
            continue
        pvname = item.strip()
        oname = "signal_{}".format(len(obj_dict))
        obj = EpicsSignal(pvname, name=oname)
        obj_dict[oname] = obj
    return obj_dict

obj_dict = connect_pvlist(pvlist)

Look at that dictionary...

In [4]:
# obj_dict

## Prepare RunEngine and Broker

We have all the tools in place so let's take a snapshot of these PVs.  First we set up the RunEngine and databroker using our workstation's mongodb configuration.

In [5]:
RE = RunEngine({})
# assuming we don't have a named configuration file available
# see: http://nsls-ii.github.io/databroker/tutorial.html#get-a-broker
cfg = {
    'metadatastore' : {
        'module' : 'databroker.headersource.mongo',
        'class' : 'MDS',
        'config' : {
            'host': 'localhost',
             'port': 27017,
             'database': 'metadatastore-production-v1',
             'timezone': 'US/Central'},
    },
    'assets' : {
        'module' : 'databroker.assets.mongo',
        'class' : 'Registry',
        'config': {
            'host': 'localhost', 
            'port': 27017, 
            'database': 'filestore-production-v1'}
    },
}
db = Broker.from_config(cfg)
RE.subscribe(db.insert)

0

## take a snapshot

Now, take a snapshot.

In [6]:
RE(snapshot(obj_dict.values(), md=dict(purpose="jupyter notebook demo")), print)

RuntimeError: This event loop is already running

start {'uid': '80d8ccc9-2627-4302-8794-17f9e3388887', 'time': 1545255514.9400032, 'scan_id': 1, 'plan_type': 'generator', 'plan_name': 'snapshot', 'plan_description': 'archive snapshot of ophyd Signals (usually EPICS PVs)', 'purpose': 'jupyter notebook demo', 'hints': {}, 'iso8601': '2018-12-19 15:38:34.938716'}
descriptor {'run_start': '80d8ccc9-2627-4302-8794-17f9e3388887', 'time': 1545255515.2849474, 'data_keys': {'signal_2': {'source': 'PV:rpi5bf5:0:humidity:1m', 'dtype': 'number', 'shape': [], 'precision': 2, 'units': '%', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'signal_2'}, 'signal_4': {'source': 'PV:rpi5bf5:0:humidity:1m:sdev', 'dtype': 'number', 'shape': [], 'precision': 2, 'units': '%', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'signal_4'}, 'signal_1': {'source': 'PV:rpi5bf5:0:temperature', 'dtype': 'number', 'shape': [], 'precision': 1, 'units': 'C', 'lower_ctrl_limit': 0.0, 'upper_ctrl_limit': 0.0, 'object_name': 'signal_1'}, '